# 모델 배포 및 추론

아래 모델 배포는 아래의 인스턴스에서 테스트 완료 되었습니다.
- ml.p4d.24xlarge, ml.g5.12xlarge
    - ml.g5.4xlarge 는 Out of Memory 발생

## 1. 환경 구성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = "../.."
add_python_path(module_path)

python path: /home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3 is added
sys.path:  ['/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3/notebook/02-naver-news-fsdp-QLoRA', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python310.zip', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/lib-dynload', '', '/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages', '/home/ec2-user/SageMaker/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/30_fine_tune/03-fine-tune-llama3']


In [2]:
%store -r model_s3_path
print("model_s3_path: ", model_s3_path)


model_s3_path:  {'S3DataSource': {'S3Uri': 's3://sagemaker-us-east-1-057716757052/llama3-8b-naver-news-2024-07-15-05-54-5-2024-07-15-05-55-02-818/output/model/', 'S3DataType': 'S3Prefix', 'CompressionType': 'None'}}


## 2. 추론 이미지 가져오기



In [3]:
import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::057716757052:role/gen_ai_gsmoon
sagemaker bucket: sagemaker-us-east-1-057716757052
sagemaker session region: us-east-1


In [5]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  session=sess,
  version="2.0.2",
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.0.2-gpu-py310-cu121-ubuntu22.04


## 3. SageMaker Model 생성

In [6]:
instance_type = "ml.g5.12xlarge"
# instance_type = "ml.p4d.24xlarge"

if instance_type == "ml.p4d.24xlarge":
    num_GPUSs = 8
elif instance_type == "ml.g5.12xlarge":
    num_GPUSs = 4
else:
    num_GPUSs = None
    
print(f"{instance_type} and # of GPU {num_GPUSs} is set")

ml.g5.12xlarge and # of GPU 4 is set


In [7]:
from huggingface_hub import HfFolder
from sagemaker.huggingface import HuggingFaceModel

# sagemaker config

health_check_timeout = 1200 # 20 minutes

import time
sm_endpoint_name = "llama3-endpoint-{}".format(int(time.time()))
print("sm_endpoint_name: \n", sm_endpoint_name)

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model",       # Path to the model in the container
  'SM_NUM_GPUS': f"{num_GPUSs}",        # Number of GPU used per replica
  'MAX_INPUT_LENGTH': "8000",           # Max length of input text
  'MAX_TOTAL_TOKENS': "8096",           # Max length of the generation (including input text)
  'MAX_BATCH_PREFILL_TOKENS': "16182",  # Limits the number of tokens that can be processed in parallel during the generation
  'MESSAGES_API_ENABLED': "true",       # Enable the OpenAI Messages API
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  # path to s3 bucket with model, we are not using a compressed model
  # {'S3DataSource':{'S3Uri': "s3://...",'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  model_data=model_s3_path,
  image_uri=llm_image,
  env=config
)

sm_endpoint_name: 
 llama3-endpoint-1721030487


/home/ec2-user/SageMaker/.cs/conda/envs/llama3_puy310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 4. 모델 배포

In [8]:
%%time
# Deploy model to an endpoint
llm = llm_model.deploy(
  initial_instance_count=1,
  instance_type=instance_type,
  endpoint_name=sm_endpoint_name,
  container_startup_health_check_timeout=health_check_timeout, # 20 minutes to give SageMaker the time to download and merge model
)

-----------------!CPU times: user 272 ms, sys: 21 ms, total: 293 ms
Wall time: 9min 3s


## 5. 모델 추론

### Helper 함수

In [9]:
from scripts.inference_util import (
    print_json,
    create_messages_parameters,
)

### SageMaker Predictor 로 추론

In [10]:

# system_prompt = "You are a helpful assistant."
# user_prompt = "Tell me something about Amazon SageMaker?"
# messages, parameters = create_messages_parameters(system_prompt = system_prompt, user_prompt = user_prompt)   

In [11]:
# chat = llm.predict({"messages" :messages, **parameters})
# print_json(chat)
# print(chat["choices"][0]["message"]["content"].strip())
# print_json(chat["choices"][0]["message"]["content"].strip())



### Boto3 InvokeEndpoint() 호출

In [12]:
import json

from scripts.inference_util import (
    create_boto3_request_body,
    invoke_endpoint_sagemaker,
    print_ww
)

In [13]:
# system_prompt = "You are a helpful assistant and write only in English"
# user_prompt = "How to make cake?"
# request_body = create_boto3_request_body(system_prompt=system_prompt, user_prompt=user_prompt)
# request_body

In [14]:
# import time
# s = time.perf_counter()

# # sm_endpoint_name = "llama3-endpoint-mnist-1719625657"
# response = invoke_endpoint_sagemaker(endpoint_name = sm_endpoint_name, 
#                          pay_load = request_body)    

# elapsed_async = time.perf_counter() - s

# print(f"elapsed time: {round(elapsed_async,3)} second")
# parsed_data = json.loads(response)
# print(json.dumps(parsed_data, indent=4, ensure_ascii=False))
# answer = parsed_data["choices"][0]["message"]["content"].strip()
# print_json(answer)
# print_ww(answer)


## 요약 추론

In [15]:
%store -r full_test_data_json

In [16]:

from scripts.inference_util import (
    get_message_from_dataset,
    extract_system_user_prompt,
    run_inference,
    generate_response
)

In [17]:
messages, full_test_dataset, rand_idx = get_message_from_dataset(
                                        sample_dataset_json_file = full_test_data_json, verbose=False)    
generate_response(messages, sm_endpoint_name, full_test_dataset, rand_idx)    

elapsed time: 1.652 second
**Query:**
{'messages': [{'role': 'system', 'content': 'You are an AI assistant specialized in news articles.Your role is to provide accurate summaries and insights in Korean. Please analyze the given text and provide concise, informative summaries that highlight the key goals and findings.'}, {'role': 'user', 'content': 'Please summarize the goals for journalist in this text:\n\n산업현장에서 세종텔레콤의 스마트 안전 플랫폼 솔루션 을 활용하고 있다. 세종텔레콤 제공 세종텔레콤은 태영건설에 중대재해처벌법 대응에 최적화된 스마트 안전 플랫폼 솔루션 납품 계약을 체결했다고 4일 밝혔다. 우선 태영건설 전국 산업현장에 스마트 안전 솔루션 500대 납품 계약을 체결했고 추가 구축을 협의 중이다. 지난 1월 본격 시행된 중대재해처벌법은 산업현장에서 인명사고 발생 시 경영진이나 법인에게 책임을 물을 수 있도록 규정한 법이다. 해당 법은 안전 보건 관련 관리상의 조치 구축을 의무화하고 있으나 기업의 한정적 자원과 부족한 인력 문제로 어려움을 겪고 있다. 세종텔레콤의 스마트 안전 플랫폼 솔루션은 출입관리부터 CCTV 가스탐지 각종 센서 등을 하나로 통합해 현장을 종합 관리할 수 있다. LBS 위치기반 IoT 사물인터넷 등 스마트 기술을 융합했다. 안전 관리 담당자는 각 현장마다 설치된 카메라 및 CCTV 개소별 센서와 통신 인프라를 통해 현장 정보를 실시간으로 확인하고 비상 상황 시에는 전체 현장 또는 해당 구역 상황실 시스템이나 모바일로 근로자에게 안전 조치사항을 지시할 수 있다. 이와 함께 타워크레인에 설치한 360도 카메라를 

# 엔드포인트 삭제

In [18]:
llm.delete_model()
llm.delete_endpoint()